In [1]:
import os
import numpy as np
from astropy import units as u
from astroquery.vizier import Vizier
from astropy.table import Table, join,vstack
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
import warnings
from astropy.utils.metadata import MergeConflictWarning
import time
import logging
from typing import List
import yaml

In [8]:
class ClusterDias:
    def __init__(self, name:str) -> None:
        #download the catalog if not already in the cwds
        if os.path.exists('./dias2021.tsv'):
            dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
        else:
            Vizier.ROW_LIMIT = -1
            dias2021 = Vizier.get_catalogs(catalog="J/MNRAS/504/356/table12")[0]
            mask_logage = dias2021['logage'] < 7.7
            mask_plx = dias2021['Plx'] > 0.3
            dias2021 = dias2021[mask_logage & mask_plx]
        #select the row from Dias catalog
        cluster_row = dias2021[dias2021['Cluster'] == name][0]
        #cluster parameters from the row
        self.name = name
        self.all = cluster_row
        self.r50 = cluster_row['r50']*u.deg
        self.N = cluster_row['N']
        self.skycoord = SkyCoord(ra=cluster_row['RA_ICRS']*u.deg,
                                 dec=cluster_row['DE_ICRS']*u.deg,
                                 distance=cluster_row['Dist']*u.pc,
                                 pm_ra_cosdec=cluster_row['pmRA']*u.mas/u.yr,
                                 pm_dec=cluster_row['pmDE']*u.mas/u.yr,
                                 obstime=(Time('J2000')+1*u.Myr))
        self.ra = self.skycoord.ra
        self.dec = self.skycoord.dec
        self.distance,self.e_distance = self.skycoord.distance,cluster_row['e_Dist']*u.pc
        self.pm_ra_cosdec, self.e_pm_ra_cosdec = self.skycoord.pm_ra_cosdec,cluster_row['e_pmRA']*u.mas/u.yr
        self.pm_dec, self.e_pm_dec = self.skycoord.pm_dec, cluster_row['e_pmDE']*u.mas/u.yr
        self.Av, self.e_Av = cluster_row['Av']*u.mag,cluster_row['e_Av']*u.mag
        self.logage, self.e_logage = cluster_row['logage'],cluster_row['e_logage']
        self.FeH, self.e_FeH = cluster_row['__Fe_H_'],cluster_row['e__Fe_H_']
        self.RV, self.e_RV = cluster_row['RV'],cluster_row['e_RV']
        self.NRV = cluster_row['NRV']
        self.members = (Table.read(f'./Clusters_Dias/{self.name}.dat', format='ascii.tab'))[2:] #first two rows removed
        self.members_list = list(self.members['Source'].astype(np.int64))
    def full_catalog() -> Table:
        dias2021 = Vizier.get_catalogs(catalog="J/MNRAS/504/356/table12")[0]
        return dias2021

class ClusterCG:
    def __init__(self, name:str) -> None:
        self.name = name
        #download the catalog if not already in the cwds
        if os.path.exists('./CG2020.tsv'):
            CG2020 = Table.read('CG2020.tsv', format='ascii.ecsv')
        else:
            Vizier.ROW_LIMIT = -1
            CG2020 = Vizier.get_catalogs(catalog="J/A+A/633/A99/table1")[0]
        
        cluster_row = CG2020[CG2020['Cluster'] == name][0]
        self.r50 = cluster_row['r50']*u.deg

    def full_catalog() -> Table:
        CG2020 = Vizier.get_catalogs(catalog="J/A+A/633/A99/table1")[0]
        return CG2020
        
class Cluster:
    def __init__(self, name:str) -> None:
        clusterDias = ClusterDias(name=name)
        clusterCG = ClusterCG(name=name)
        self.name = name
        self.skycoord = clusterDias.skycoord
        self.ra = clusterDias.skycoord.ra
        self.dec = clusterDias.skycoord.dec
        self.distance = clusterDias.skycoord.distance
        self.r50 = clusterCG.r50
        self.r50_phy = np.tan(self.r50) * self.distance
        self.search_arcmin = search_arcmin(self.distance, self.r50)
        
    def stars_in_region(self):
        stars_in_region_path =  f'{self.name}/{self.name}_stars_in_region.tsv'
        
        if os.path.exists(stars_in_region_path):
            stars_in_region = Table.read(stars_in_region_path, format='ascii.ecsv')
        else:
            stars_in_region = self.get_stars_in_region()
            stars_in_region.write(stars_in_region_path, format='ascii.ecsv')
        return stars_in_region

    def get_stars_in_region(self) -> Table:
        c = ClusterDias(self.name).skycoord
        t1 = searchDR3(c,self.search_arcmin)
        t2 = searchDR3_dist(c,self.search_arcmin)
        t3 = merge_gaia_tables(t1,t2)
        t3.sort('Gmag')
        return t3





In [3]:
def read_yaml_file(file_path):
    '''
    Read the configuration file for the rest of the code. 
    This contains the various parameters for the code to run.
    '''
    with open(file_path, 'r') as yaml_file:
        config = yaml.safe_load(yaml_file)
    return config
config = read_yaml_file('config2.yaml')


# Configure logging
def search_arcmin(distance, radius:Angle, extra=config['Cluster']['search_extent']):
    theta = radius
    D = distance
    r = np.tan(theta) * D #physical radius
    search_arcmin = np.arctan((r + extra * u.pc) / D)
    search_arcmin = search_arcmin.to(u.arcminute)
    return search_arcmin.round(3)


def searchDR3(skycoordinate: SkyCoord, radius: Angle) -> Table:
    start_time = time.time()
    print(f"Searching in Gaia DR3 I/355/gaiadr3 {radius} around {skycoordinate.ra, skycoordinate.dec, skycoordinate.distance}")

    filters = {'Gmag': '<17', 'Plx': '>0.3'}
    stars_fromDR3 = Vizier(columns=["*", "+_r"], row_limit=-1).query_region(
        skycoordinate, 
        radius=radius, 
        catalog="I/355/gaiadr3",
        column_filters=filters
    )[0]
    
    stars_fromDR3['SkyCoord1'] = SkyCoord(
        ra=stars_fromDR3['RA_ICRS'],
        dec=stars_fromDR3['DE_ICRS'],
        distance=(stars_fromDR3['Plx']).to(u.pc, equivalencies=u.parallax()),
        pm_ra_cosdec=stars_fromDR3['pmRA'],
        pm_dec=stars_fromDR3['pmDE'],
        obstime=(Time('J2000') + 1 * u.Myr)
    )

    end_time = time.time()
    print(f"found {len(stars_fromDR3):,} sources in {end_time - start_time:.2f} seconds")
    return stars_fromDR3

def searchDR3_dist(skycoordinate: SkyCoord, radius: Angle) -> Table:
    start_time = time.time()
    print(f"Searching in Gaia DR3 distances I/352/gedr3dis {radius} around {skycoordinate.ra, skycoordinate.dec, skycoordinate.distance}")


    stars_fromDR3_dist = Vizier(columns=["*", "+_r"], row_limit=-1).query_region(
        skycoordinate, 
        radius=radius, 
        catalog="I/352/gedr3dis"
    )[0]
    
    stars_fromDR3_dist['SkyCoord2'] = SkyCoord(
        ra=stars_fromDR3_dist['RA_ICRS'],
        dec=stars_fromDR3_dist['DE_ICRS'],
        distance=(stars_fromDR3_dist['rgeo']),
        obstime=(Time('J2000') + 1 * u.Myr)
    )

    end_time = time.time()
    print(f"found {len(stars_fromDR3_dist):,} sources in {end_time - start_time:.2f} seconds")
    filtered_catalog = stars_fromDR3_dist[skycoordinate.separation_3d(stars_fromDR3_dist['SkyCoord2']) < skycoordinate.distance*config['Cluster']['distance_tolerance']]
    return filtered_catalog

def merge_gaia_tables(stars_fromDR3: Table, stars_fromDR3_dist: Table) -> Table:
    start_time = time.time()
    print("Starting merge of DR3 and distance catalog data")

    warnings.simplefilter('ignore', MergeConflictWarning)
    merged = join(stars_fromDR3, stars_fromDR3_dist, keys='Source', join_type='inner')

    # Order the columns
    merged = merged['RA_ICRS_1', 'DE_ICRS_1', 'e_RA_ICRS', 'e_DE_ICRS', '_r_1',
                                       'HIP', 'TYC2', 'Source', 'rgeo', 'Plx', 'e_Plx',
                                       'pmRA', 'pmDE', 'e_pmRA', 'e_pmDE',
                                       'RUWE', 'Teff', 'logg', 'Gmag', 'BP-RP', 'BPmag', 'RPmag', 'RV', 'e_RV',
                                       'b_rgeo', 'B_rgeo', 'FG', 'e_FG', 'FBP', 'e_FBP', 'FRP', 'e_FRP', 'RAVE5', 'RAVE6']

    # Adding row for uncertainties in Gmag and BPmag and RPmag
    # values for Gaia G, G_BP, G_RP zero point uncertainties
    sigmaG_0 = 0.0027553202
    sigmaGBP_0 = 0.0027901700
    sigmaGRP_0 = 0.0037793818

    merged['e_Gmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FG'] / merged['FG'])**2 + sigmaG_0**2)
    merged['e_BPmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FBP'] / merged['FBP'])**2 + sigmaGBP_0**2)
    merged['e_RPmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FRP'] / merged['FRP'])**2 + sigmaGRP_0**2)
    merged['e_BP-RP'] = merged['e_BPmag'] + merged['e_RPmag']

    merged['SkyCoord'] = SkyCoord(
        ra=merged['RA_ICRS_1'],
        dec=merged['DE_ICRS_1'],
        distance=(merged['rgeo']),
        pm_ra_cosdec=merged['pmRA'],
        pm_dec=merged['pmDE'],
        obstime=(Time('J2000') + 1 * u.Myr)
    )

    end_time = time.time()
    print(f"{len(merged):,} sources found by merging in {end_time - start_time:.2f} seconds")
    return merged

In [10]:
cl = Cluster("Berkeley_97")
cl.stars_in_region()

Searching in Gaia DR3 I/355/gaiadr3 15.934 arcmin around (<Longitude 339.8499 deg>, <Latitude 59.0046 deg>, <Distance 2485. pc>)
found 1,197 sources in 0.15 seconds
Searching in Gaia DR3 distances I/352/gedr3dis 15.934 arcmin around (<Longitude 339.8499 deg>, <Latitude 59.0046 deg>, <Distance 2485. pc>)
found 17,607 sources in 1.22 seconds
Starting merge of DR3 and distance catalog data
443 sources found by merging in 0.03 seconds


RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str12,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str16,str21,float64,float32,float32,float32,SkyCoord
339.84392032630,59.00715291679,0.0293,0.0308,0.2402,--,3995-625-1,2008383302396655488,2630.22485000,0.3652,0.0363,-2.873,-2.301,0.038,0.038,3.035,18135.5,3.3302,9.473729,0.937676,9.843719,8.906042,--,--,2393.19263000,2940.37549000,3058125.18769,951.5,1.577e+06,1154,2.171e+06,1124,,,0.002776,0.002901,0.003821,0.006722,"339.8439203263,59.00715291679,2630.22485"
339.82384023875,59.01681945374,0.0097,0.0105,1.0890,--,3995-1075-1,2008430237808841600,2657.56860000,0.3583,0.0125,-2.747,-1.911,0.013,0.013,0.986,--,--,9.840384,0.868947,10.178231,9.309284,--,--,2587.60059000,2734.51587000,2181691.83228,1048,1.159e+06,1141,1.498e+06,1215,,,0.002804,0.002988,0.003881,0.006869,"339.82384023875,59.01681945374,2657.5686"
339.78276898862,59.07086926297,0.0265,0.0264,4.4837,--,,2008431406040225664,2606.06836000,0.3659,0.0315,-3.951,-0.770,0.035,0.034,0.995,--,--,11.117977,3.769605,13.508001,9.738397,-69.25,0.34,2369.06812000,2884.74414000,672598.26706,1163,5.398e+04,605.7,1.009e+06,4436,,,0.003334,0.0125,0.006089,0.01859,"339.78276898862,59.07086926297,2606.06836"
339.83096349165,58.74634416929,0.0226,0.0224,15.5067,--,,2008365126103315328,2198.32202000,0.4344,0.0276,-2.712,-2.078,0.032,0.026,1.649,--,--,11.164767,2.093456,12.238070,10.144614,-39.86,5.08,2056.71948000,2342.02271000,644228.09039,157.3,1.739e+05,112,6.939e+05,250.8,,,0.002768,0.002876,0.0038,0.006676,"339.83096349165,58.74634416929,2198.32202"
340.11072199405,58.92710174793,0.0187,0.0209,9.3118,--,,2008379900791042048,2509.77856000,0.3757,0.0235,-5.845,-4.092,0.025,0.027,1.039,3695.2,0.5505,11.359638,3.303004,13.357115,10.054111,-89.02,0.35,2353.26758000,2706.65967000,538383.27321,458,6.203e+04,346.3,7.542e+05,1542,,,0.002906,0.006673,0.004383,0.01106,"340.11072199405,58.92710174793,2509.77856"
340.10268328507,58.97390297864,0.0097,0.0107,8.0278,--,3996-1228-1,2008380794144211712,2570.86328000,0.3525,0.0122,-2.679,-1.930,0.013,0.014,0.903,--,--,11.482301,1.201067,11.985507,10.784440,--,--,2498.03125000,2653.16870000,480868.62054,798.2,2.194e+05,725.5,3.849e+05,2491,,,0.003292,0.004547,0.007978,0.01253,"340.10268328507,58.97390297864,2570.86328"
339.74109632951,58.83600736330,0.0092,0.0096,10.6625,--,,2008378212857607680,2628.28149000,0.3736,0.0111,-4.228,-2.178,0.012,0.012,0.984,--,--,12.033627,2.239635,13.207543,10.967909,-43.07,0.24,2548.67554000,2691.39307000,289398.40372,52.27,7.119e+04,45.75,3.251e+05,80.88,,,0.002762,0.002876,0.003789,0.006665,"339.74109632951,58.8360073633,2628.28149"
339.43148475774,58.95033027843,0.0375,0.0380,13.3423,--,,2008409862483646080,2365.09619000,0.4191,0.0452,-0.075,-2.012,0.051,0.046,1.027,--,--,12.311062,4.622185,15.445667,10.823483,-33.77,0.33,2146.03052000,2611.00269000,224141.35144,321.3,9060,87.73,3.713e+05,1419,,,0.003164,0.01088,0.005612,0.01649,"339.43148475774,58.95033027843,2365.09619"
